In [3]:
%gui qt
%matplotlib qt

import numpy as np
from pprint import pprint
from importlib import reload

In [28]:
from plottr.data.datadict import DataDict

from plottr import node; reload(node)
from plottr.node import Node, NodeWidget, DataSelector

# Tool functions

In [5]:
def testdata_1d(nvals):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx, ny, nz):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1)),
        y = dict(values=yy.reshape(-1)),
        z = dict(values=zz.reshape(-1)),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z']),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z']),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'])
    )
    return d

# Basic data selection

In [ ]:
data1 = testdata_3d(3, 5, 5)
datasrc = Node()
datasrc.verbose = True
datasrc.data = data1

In [ ]:
datasel = DataSelector()
datasel.verbose = True
datasel.setInput(datasrc)

In [ ]:
datasel.dataName = 'more_data'
datasel.slices = dict(y=np.s_[::2], x=np.s_[:2:])
datasel.axesOrder = dict(x=1,z=2,y=0)

# Basic checking

In [ ]:
data1 = testdata_3d(3, 5, 5)

datasrc = Node()
datasrc.verbose = False

datasel = DataSelector()
datasel.verbose = False
datasel.setInput(datasrc)

In [ ]:
datasrc.data = data1
datasel.dataName = ['data', 'more_data']

In [ ]:
datasel.data

# Data Selector Widget

In [79]:
from PyQt5.QtWidgets import (QWidget, QVBoxLayout, QFormLayout, 
                             QComboBox, QLabel, QGroupBox, QCheckBox)
from PyQt5.QtCore import QObject, QThread, pyqtSignal, pyqtSlot


class NodeControlWidget(QWidget):
    
    def __init__(self, parent=None, **kw):
        super().__init__(parent=parent, **kw)
        

# class DataNameCheckbox(QWidget):
    
#     def __init__(self, name, parent=None):
#         super().__init__(parent=parent)
        
      

class DataSelectorWidget(NodeControlWidget):
    
    newDataStructure = pyqtSignal()
    axesChoiceUpdated = pyqtSignal(str, str)
    
    def __init__(self, parent=None, node=None, **kw):
        super().__init__(parent=parent, **kw)
        
        # this is more general!
        if node is None:
            self.node = DataSelector()
        else:
            self.node = node
        self.node.sourceDataUpdated.connect(self.dataUpdate)
        
        # Axes selection
        self._axesOptions = []
        self.xCombo = QComboBox()
        self.yCombo = QComboBox()
        self.xCombo.currentTextChanged.connect(
            lambda choice: self.selectAx('x', choice))
        self.yCombo.currentTextChanged.connect(
            lambda choice: self.selectAx('y', choice))
        
        axLayout = QFormLayout()
        axLayout.addRow('x-axis', self.xCombo)
        axLayout.addRow('y-axis', self.yCombo)
        axGroup = QGroupBox('Axis selection')
        axGroup.setLayout(axLayout)
        
        # Data fields
        self._dataOptions = {}
        
        
        self.dataLayout = QFormLayout()
        dataGroup = QGroupBox('Data selection')
        dataGroup.setLayout(self.dataLayout)
        
        
        mainLayout = QVBoxLayout(self)
        mainLayout.addWidget(axGroup)
        mainLayout.addWidget(dataGroup)
        
        self.newDataStructure.connect(self.updateAxOptions)
        self.newDataStructure.connect(self.updateDataOptions)
        
        
    def dataUpdate(self, sourceName):
        # TODO: is this too agressive?
        print("got new data from {}".format(sourceName))
        srcData = self.node.getInputData()
        newAxes = self.node.axesList(srcData)
        if newAxes != self._axesOptions:
            self._axesOptions = newAxes
            self.newDataStructure.emit()
    
    @pyqtSlot()
    def updateAxOptions(self):
        axOptions = [''] + self._axesOptions
        for combo in self.xCombo, self.yCombo:
            combo.clear()
            combo.addItems(axOptions)
    
    def _deleteDataOptions(self, names):
        for name in names:
            v = self._dataOptions[name]
            self.dataLayout.removeWidget(v['widget'])
            self.dataLayout.removeWidget(v['label'])
            v['widget'].deleteLater()
            v['label'].deleteLater()
            del self._dataOptions[name]
    
    @pyqtSlot()
    def updateDataOptions(self):
        srcData = self.node.getInputData()
        
        delete = []
        for k, v in self._dataOptions.items():
            delete.append(k)
        self._deleteDataOptions(delete)
        
        for n in srcData.dependents():
            if n not in self._dataOptions:
                lbl = QLabel(n)
                chk = QCheckBox()
                self._dataOptions[n] = dict(label=lbl, widget=chk)
                self.dataLayout.addRow(lbl, chk)
            
    def selectAx(self, dimName, axName):
        print("New selection for '{}': '{}'".format(dimName, axName))
        emit = True
        
        if dimName == 'x':
            if axName != '' and axName == self.yCombo.currentText():
                emit = False
                self.yCombo.setCurrentText('')
        elif dimName == 'y':
            if axName != '' and axName == self.xCombo.currentText():
                emit = False
                self.xCombo.setCurrentText('')
                
        order = {}
        curx = self.xCombo.currentText()
        cury = self.yCombo.currentText()
        if curx != '':
            order[curx] = 0
        if cury != '':
            order[cury] = 1
        
        if emit:
            self.node.axesOrder = order
            self.axesChoiceUpdated.emit(curx, cury)
            print('New axes choices: {}, {}'.format(curx, cury))
        
    

In [80]:
data3d = testdata_3d(3, 5, 5)
data1d = testdata_1d(10)

datasrc = Node()
datasrc.data = data3d

w = DataSelectorWidget()
w.show()
w.node.verbose = True
w.node.setInput(datasrc)

<plottr.node.DataSelector object at 0x000001C0183B8318> data set from input
got new data from input
New selection for 'x': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
New selection for 'y': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}



In [83]:
datasrc.data = data1d

<plottr.node.DataSelector object at 0x000001C0183B8318> data set from input
got new data from input
New selection for 'x': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
New selection for 'x': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
New selection for 'y': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
New selection for 'y': ''
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}

New axes choices: , 
processed data: <plottr.node.DataSelector object at 0x000001C0183B8318>
{}



# Updating a 1D plot

In [ ]:
data1 = testdata_1d(21)
datasrc = Node()
datasrc.data = data1

lineplotdata = LinePlotData()
lineplotdata.setInput(datasrc)

lp = LinePlot(None)
lp.setInput(lineplotdata)
lp.show()

In [ ]:
lineplotdata.xaxisName = 'x'
lineplotdata.traceNames = ['z']